In [ ]:
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import pandas as pd
from sklearn.preprocessing import LabelEncoder as le
from collections import defaultdict


In [ ]:
df_raw = pd.read_excel("output/resampled_df_10_min.xlsx", index_col=[0])

In [ ]:
start_date = pd.to_datetime(f"2023-01-01 00:00:00")
end_date = pd.to_datetime(f"2023-05-01 23:50:00")
training_window_size = 7
horizon_size = 7
model_features = ["day", "weekday", "hour", "window_block"] # Day = day of the month (0-31), hour = hour of the day (0-24), weekday = day in the week (0-7), window_block = window block in the hour (0-5)

baseline_performance = defaultdict(dict)

In [ ]:
df = df_raw[df_raw["time"].between(start_date, end_date)].copy()

label_encoder = le()
df.location = label_encoder.fit_transform(df.location)

In [ ]:
df["day"] = df["time"].dt.day
df["weekday"] = df["time"].dt.dayofweek
df["hour"] = df["time"].dt.hour
df["window_block"] = ((df['time'].dt.minute * 60 + df['time'].dt.second) // 600).astype(int)


In [ ]:
df.head(10)

In [ ]:
train_start_date = start_date
train_end_date = train_start_date + pd.Timedelta(days=training_window_size-1, hours=23, minutes=50)
test_start_date = train_end_date + pd.Timedelta(minutes=10)
test_end_date = test_start_date + pd.Timedelta(days=horizon_size-1, hours=23, minutes=50)

train_mask = df["time"].between(train_start_date, train_end_date)
test_mask = df["time"].between(test_start_date, test_end_date)

# Split the data into train and test sets
X_train = df.loc[train_mask, model_features]
y_train = df.loc[train_mask, "location"]
X_test = df.loc[test_mask, model_features]
y_test = df.loc[test_mask, "location"]

print(f"Training: {train_start_date}-{train_end_date}, testing: {test_start_date}-{test_end_date}.")

In [ ]:
training_data = df.loc[train_mask]
testing_data = df.loc[test_mask]
most_common_locations = training_data.groupby(model_features)['location'].apply(lambda x: x.value_counts().idxmax()).reset_index()

In [ ]:
result_df = testing_data.merge(most_common_locations, how="left", left_on=model_features, right_on=model_features)

features_to_use = model_features[1:]
while result_df['location_y'].isna().sum() > 0:
    print('nan > 0, now trying with features: ', features_to_use)
    most_common_locations = training_data[["location"] + features_to_use].groupby(features_to_use)['location'].apply(lambda x: x.value_counts().idxmax()).reset_index()
    result_df = testing_data.merge(most_common_locations, how="left", left_on=features_to_use, right_on=features_to_use)
    features_to_use = features_to_use[1:]  # Remove the first element to exclude it from the next merge

predictions = result_df.location_y.values.tolist()



In [ ]:
for d in range(horizon_size):
    # Then, evaluate the baseline's predictions and store acc in self.baseline_performance
    this_day_predictions = predictions[d*144:(d+1)*144]
    this_day_actual_values = y_test[d*144:(d+1)*144]
    acc = accuracy_score(this_day_actual_values, this_day_predictions)
    print(f"Acc of baseline: {acc}")

In [12]:
import DataLoader as DL

# Initialize parameters.
data_source = "google_maps"  # Can be either 'google_maps' or 'routined'.
# hours_offset is used to offset the timestamps to account for timezone differences. For google maps, timestamp comes in GMT+0
# which means that we need to offset it by 2 hours to make it GMT+2 (Dutch timezone). Value must be INT!
hours_offset = 2 # Should be 0 for routined and 2 for google_maps. 
# begin_date and end_date are used to filter the data for your analysis.
begin_date = "2010-01-01"
end_date = "2023-12-30"  # End date is INclusive! 
# FRACTION is used to make the DataFrame smaller. Final df = df * fraction. This solves memory issues, but a value of 1 is preferred.
fraction = 1
# For the heatmap visualization we specify a separate begin_date and end_date (must be between begin_date and end_date).
# For readiness purposes, it it suggested to select between 2 and 14 days.
heatmap_begin_date = "2023-01-20"
heatmap_end_date = "2023-05-28"  # End date is INclusive! Choose a date that lies (preferably 2 days) before end_date to avoid errors. 
# For the model performance class we need to specify the number of training days (range) and testing horizon (also in days)
training_window_size = 100
horizon_size = 30
window_step_size = 1
outputs_folder_name = f"ankit1-{training_window_size}-{horizon_size}-{window_step_size}" # All of the outputs will be placed in output/outputs_folder_name

df = DL.load_data(
    data_source,
    begin_date,
    end_date,
    fraction,
    hours_offset,
    outputs_folder_name=outputs_folder_name,
    verbose=True,
    perform_eda=True
)

Message (data loader): Since HOUR_OFFSET > 0, we offset the timestamps with 2 hours.
Message (data loader): Loaded google_maps data from 2010-01-01 to 2023-12-30 with a fraction of 1. Length of data: 552928
Message (data loader): First record in dataset is from 2014-09-04 15:17:59 and last record is from 2022-09-01 11:49:06
Message (data loader): Performing EDA, saving plots at output/ankit1-100-30-1
